In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats import power, proportion
import pickle
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh import palettes
output_notebook()
%matplotlib inline

In [ ]:
pwr = power.NormalIndPower()

In [ ]:
n = 101

p1 = np.linspace(0, 1, n)
p2 = np.linspace(0, 1, n)
nobs = np.zeros((n, n)) + np.nan

for i, p1_ in enumerate(p1):
    for j, p2_ in enumerate(p2):
        h = proportion.proportion_effectsize(p1_, p2_)
        if i == j:
            continue
        try:
            nobs[i, j] = pwr.solve_power(
                effect_size=h, alpha=0.05, power=0.8, 
                ratio=1.0, alternative='two-sided'
            )
        except:
            pass

In [ ]:
p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
p.x_range.range_padding = p.y_range.range_padding = 0
p.image([np.log10(nobs)], 0, 0, 1, 1, palette="Inferno256")
show(p)  # open a browser

In [ ]:
inferno256 = palettes.inferno(256)

mn, mx = np.nanmin(np.log10(nobs)), np.nanmax(np.log10(nobs))
cmap = np.linspace(mn, mx, 256)

In [ ]:
color = []
x, y, dp = [], [], []
for i, p1_ in enumerate(p1):
    for j, p2_ in enumerate(p2):
        z = np.log10(nobs[i, j])
        if np.isnan(z):
            c = '#ffffff'
        else:
            idx = (z >= cmap).sum()-1
            c = inferno256[idx]
        color.append(c)
        x.append(p1_)
        y.append(p2_)
        dp.append(p2_ - p1_)

In [ ]:
src=dict(
    xname=x,
    yname=y,
    dp=dp,
    colors=color,
    count=nobs.flatten()
)

tooltips = [
    ('Control', '@xname{0.00}'), 
    ('Test', '@yname{0.00}'), 
    ('Test - Control', '@dp{0.00}'),
    ('Sample Size', '@count{0}')
]

p = figure(
    x_axis_location="above", tools="hover,crosshair",
    x_range=(0, 1), y_range=(0, 1),
    tooltips = tooltips
)

p.plot_width = 600
p.plot_height = 600
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "12pt"
p.xaxis.axis_label = 'Control Probability'
p.yaxis.axis_label = 'Test Probability'


p.rect('xname', 'yname', 0.01, 0.01, source=src,
       color='colors', line_color='colors',
       hover_line_color='black', hover_color='colors')

show(p) # show the plot

In [ ]:
# add colorbar with log scale